<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Optuna_for_Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install optuna

In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define the neural network
class SimpleNet(nn.Module):
    def __init__(self, hidden_size):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28*28, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the objective function
def objective(trial):
    hidden_size = trial.suggest_int('hidden_size', 16, 128)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)

    model = SimpleNet(hidden_size)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('.', train=True, download=True, transform=transforms.ToTensor()),
        batch_size=32, shuffle=True)

    model.train()
    for epoch in range(2):  # Using a small number of epochs for demonstration
        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.view(-1, 28*28)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

    return loss.item()

# Create and optimize the Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

print('Best trial:', study.best_trial.params)